# About this notebook

The following notebook consolidates all the data checked in and puts it into a 5-table format described by the profs slides.

In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
labeled_entity_df = pd.read_csv('entity_list_with_labels.tsv', index_col=False, sep='\t')
print(labeled_entity_df.shape)
labeled_entity_df.head()

(190056, 3)


,term,label,count
0,LinkedIn Corporation Initial,ORG,3
1,May 2010,DATE,104
2,9 years ago,DATE,13
3,2010-05,DATE,3
4,Platform Web Browser,ORG,3


In [3]:
mikes_nodes_df = pd.read_csv('../scraping/nodes.tsv', index_col=False, sep='\t')
print(mikes_nodes_df.shape)
mikes_nodes_df.head()

(1034, 1)


,nodes
0,(Cork newspaper)
1,(magazine)
2,500
3,.NET Foundation
4,1&1 Drillisch


In [4]:
with open('../scraping/kevin_scrape/result_kk.json', 'r') as f:
    kevins_json = json.loads(f.read())
list(kevins_json.items())[:1]

[('LinkedIn',
  {'naics': '541511',
   'headcount': ['20547'],
   'Type of business': 'Subsidiary',
   'Type of site': 'Social networking service',
   'Available in': 'Multilingual (24)',
   'Founded': 'December 28, 2002; 16 years ago (2002-12-28)Mountain View, California, U.S.',
   'Headquarters': 'Sunnyvale, California, U.S.',
   'Area served': 'Worldwide',
   'Founder(s)': 'Reid HoffmanAllen BlueKonstantin GuerickeEric LyJean-Luc Vaillant',
   'CEO': 'Jeff Weiner',
   'Key people': 'Shannon Brayton (CMO) Christina Hall (CPO, SVP) Steve Sordello (CFO, SVP)',
   'Industry': 'Internet',
   'Products': 'SlideShare',
   'Employees': '15,000 (2019)',
   'Parent': 'Microsoft Corporation',
   'Subsidiaries': 'LearningConnectifier',
   'Website': 'www.linkedin.com',
   'Alexa rank': '58 (November 2019[update])[1]',
   'Advertising': 'Google, AdSense',
   'Registration': 'Required',
   'Users': '630 million members (June 2019)',
   'Launched': 'May 5, 2003; 16 years ago (2003-05-05)',
   'Cur

In [5]:
kevin_entities = [key for key in kevins_json]
print(len(kevin_entities))
kevin_entities[:5]

7


['LinkedIn', 'Lynda.com', 'ZipRecruiter Inc', 'Indeed.com', 'SlideShare']

# Entity comparison

How do the entity lists compares? Which entities appear in some and not others?

In [6]:
for k in kevin_entities:
    if not any(labeled_entity_df.term == k):
        print("Missing:", k)

Missing: ZipRecruiter Inc


In [7]:
mask = labeled_entity_df.term.apply(lambda x: 'ZipRecruiter' in str(x))
labeled_entity_df[mask]

,term,label,count
85357,ZipRecruiter,PERSON,1


Might as well fix this row...

In [8]:
labeled_entity_df.loc[mask, 'label'] = 'ORG'
labeled_entity_df[mask]

,term,label,count
85357,ZipRecruiter,ORG,1


In [9]:
%%time
target = labeled_entity_df.term.str.lower()
mask = mikes_nodes_df.nodes.apply(lambda k: not any(target == k.lower()))
cnt = np.sum(mask)
print("% of mikes_nodes_df not in labeled_entity_df: {}%".format(cnt/len(mikes_nodes_df)*100))

% of mikes_nodes_df not in labeled_entity_df: 38.49129593810445%
Wall time: 21.3 s


In [10]:
mikes_nodes_df[mask].sample(10, random_state=42)

,nodes
477,MP3
1028,telephony
100,Business relations
503,Meerkat (app)
213,Donya Labs AB
186,Czech language
972,Wretch (website)
216,Dr Ben Medlock
544,Microsoft Ignite
268,Expedia Global Partner Solutions


# Consolidating entities

It seems that `mikes_nodes_df` has at least some legitimate entities that are not otherwise represented. I'll add these to the list of nodes, with the labels ommitted. We'll need to revisit how to get the labels in the future. One possibility is to use a model like word2vec to cluster labeled nodes, and then assign unlabeled nodes based on the cluster they're closest to.

In [11]:
tmp_df = pd.DataFrame(data={"term": mikes_nodes_df[mask].nodes.value_counts().index,
                            "label": [""]*len(mikes_nodes_df[mask].nodes.value_counts()),
                            "count": mikes_nodes_df[mask].nodes.value_counts()}).reset_index(drop=True)
tmp_df.head()

,term,label,count
0,CEO,,11
1,Chairman,,7
2,supervisory board,,4
3,Microsoft Ignite,,1
4,Champagne (advertisement),,1


In [12]:
print(len(labeled_entity_df))
labeled_entity_df = labeled_entity_df.append(tmp_df)
print(len(labeled_entity_df))

190056
190435


In [13]:
# Adding the counts to the labeled_entity_df
for term, cnt in mikes_nodes_df[~mask].nodes.value_counts().items():
    tmask = labeled_entity_df.term == term
    labeled_entity_df.loc[tmask, 'count'] += cnt

For the present NodeLabels tables I'll only output the most common entities, removing some node types that I don't think we're particularly interested in.

The meaning of the entity labels is given on this page: https://spacy.io/api/annotation#named-entities

In [14]:
labeled_entity_df = labeled_entity_df.sort_values(by=['count'], ascending=False)
labeled_entity_df.head()

,term,label,count
63,Microsoft,ORG,9334
961,ISBN,ORG,3407
28,first,ORDINAL,3335
15,2,CARDINAL,3226
11,1,CARDINAL,3185


In [15]:
labeled_entity_df['count'].quantile([0.5, 0.9])

0.5    1.0
0.9    4.0
Name: count, dtype: float64

## Naics codes

I'll also add in the NAICS codes specifically as nodes. Extracted from Ashok's current graph db with the query:

    MATCH (a:NAICS) RETURN id(a) as id, a.name as node, "NAICS" as label

In [16]:
naics_nodes = pd.read_csv('naics.csv')
naics_nodes.head()

,id,node,label
0,1,541511,NAICS
1,7,518210,NAICS
2,10,541810,NAICS
3,13,NoNAICS,NAICS
4,18,561311,NAICS


In [17]:
# NoNAICS really models the absence of a relationship.
# Could we get rid of it? If you want to keep it comment the line below.
naics_nodes = naics_nodes[naics_nodes.node != 'NoNAICS']
naics_nodes = pd.DataFrame(data={"term": naics_nodes.node,
                                 "label": naics_nodes.label,
                                 "count": [1]*len(naics_nodes)})
naics_nodes.head()

,term,label,count
0,541511,NAICS,1
1,518210,NAICS,1
2,541810,NAICS,1
4,561311,NAICS,1


In [18]:
excluded_types = ['CARDINAL', 'ORDINAL', 'DATE', 'TIME']
mask = (~labeled_entity_df.label.isin(excluded_types)) & (labeled_entity_df['count'] >= 2)
print(np.sum(mask))
labeled_entity_df[mask].append(naics_nodes).drop('count', axis=1).to_csv('5tables/NodeLabels.tsv', index=False, sep='\t')

40907


In [19]:
labeled_entity_df.append(naics_nodes).drop('count', axis=1).to_csv('5tables/NodeLabels_complete.tsv', index=False, sep='\t')

In [20]:
NodeLabels_df = pd.read_csv('5tables/NodeLabels.tsv', index_col=False, sep='\t')
NodeLabels_df.head(3)

,term,label
0,Microsoft,ORG
1,ISBN,ORG
2,China,GPE


# Edge lists

Following section consolidates the edge lists. Right now this includes the edge list I generated and the edge list implicitly used by Ashok's script. In order to extract Ashok's implicit edge list I actually created a graph by running his script, and run the following cypher query against it:

    MATCH (a)-[e]->(b) RETURN id(e) as id, a.name as begin, b.name as end, type(e) as label

In [21]:
ashok_edges = pd.read_csv('ashok_edges.csv', index_col='id')
ashok_edges = ashok_edges.sort_values(by='id')
print(len(ashok_edges))
ashok_edges.head()

30


,begin,end,label
id,,,
0,541511,LinkedIn,NAICSCode
1,ACQ2,LinkedIn,Acquired
2,LinkedIn,Reid Hoffman,Founded
3,LinkedIn,Konstantin Guericke,Founded
4,LinkedIn,SlideShare,Produces


It seems like the direction of some edges is reversed, e.g. "LinkedIn-Founded->Reid Hoffman". Some are in expected order, e.g. "LinkedIn-Produces->SlideShare". In the future we can try to correct for this.

In [22]:
mikes_edges = pd.read_csv('../scraping/edges.tsv', sep='\t', index_col=False)
print(len(mikes_edges))
mikes_edges.head()

276


,start,edge_type,end
0,1&1 Drillisch,Produces,Mobile virtual network operator
1,ADVA Optical Networking,Produces,Carrier Ethernet access products
2,ADVA Optical Networking,Produces,Network management
3,ADVA Optical Networking,Produces,Wavelength division multiplexers
4,ADVA Optical Networking,Produces,software


In [23]:
connectivity_df = pd.DataFrame(data={"FromNode": mikes_edges.start,
                                     "ToNode": mikes_edges.end,
                                     "label": mikes_edges.edge_type,
                                     "origin": ["mikes_edges"]*len(mikes_edges),
                                     "originalIndex": mikes_edges.index})
connectivity_df.head()

,FromNode,ToNode,label,origin,originalIndex
0,1&1 Drillisch,Mobile virtual network operator,Produces,mikes_edges,0
1,ADVA Optical Networking,Carrier Ethernet access products,Produces,mikes_edges,1
2,ADVA Optical Networking,Network management,Produces,mikes_edges,2
3,ADVA Optical Networking,Wavelength division multiplexers,Produces,mikes_edges,3
4,ADVA Optical Networking,software,Produces,mikes_edges,4


In [24]:
connectivity_df = connectivity_df.append(
    pd.DataFrame(data={"FromNode": ashok_edges.begin,
                       "ToNode": ashok_edges.end,
                       "label": ashok_edges.label,
                       "origin": ["ashok_edges"]*len(ashok_edges),
                       "originalIndex": ashok_edges.index})).reset_index(drop=True)
connectivity_df.index.name = 'EdgeID'
connectivity_df.head().append(connectivity_df.tail())

,FromNode,ToNode,label,origin,originalIndex
EdgeID,,,,,
0,1&1 Drillisch,Mobile virtual network operator,Produces,mikes_edges,0
1,ADVA Optical Networking,Carrier Ethernet access products,Produces,mikes_edges,1
2,ADVA Optical Networking,Network management,Produces,mikes_edges,2
3,ADVA Optical Networking,Wavelength division multiplexers,Produces,mikes_edges,3
4,ADVA Optical Networking,software,Produces,mikes_edges,4
301,"Robert Hohman, Rich Barton, Tim Besse",Glassdoor,Founded,ashok_edges,41
302,"Robert Hohman, Christian Sutherland-Wong, Ryan...",Glassdoor,Iskeyto,ashok_edges,42
303,NoNAICS,Connectifier,NAICSCode,ashok_edges,43
304,John JersinBenjamin McCann,Connectifier,Founded,ashok_edges,44


Before writing out the edge list, I'll filter it so it only refers to entities in NodeLabels.tsv.

In [25]:
print(len(connectivity_df))
connectivity_mask = connectivity_df.apply(
    lambda row: (row.FromNode in NodeLabels_df.term.values) and (row.ToNode in NodeLabels_df.term.values),
    axis=1)
print(np.sum(connectivity_mask))

306
174


In [26]:
connectivity_df[connectivity_mask].drop(['origin', 'originalIndex', 'label'], axis=1).to_csv('5tables/Connectivity.tsv', sep='\t')

# Edge labels

Edge labels are easy to extract from what we've already got.

In [27]:
connectivity_df[connectivity_mask].drop(['origin', 'originalIndex', 'FromNode', 'ToNode'], axis=1).to_csv('5tables/EdgeLabels.tsv', sep='\t')

# Edge properties

Only Ashok has added edge properties so far. They are extracted from the existing graph as follows:

    MATCH ()-[e]->() RETURN id(e) as id, e as props
    

In [28]:
with open("ashok_edge_props.json", "rb") as f:
    ashok_edge_props = json.loads(f.read())

In [29]:
print(len(ashok_edge_props))
ashok_edge_props[:3]

30


[{'id': 23,
  'props': {'identity': 23,
   'start': 29,
   'end': 0,
   'type': 'Owns',
   'properties': {}}},
 {'id': 22,
  'props': {'identity': 22,
   'start': 28,
   'end': 0,
   'type': 'Iskeyto',
   'properties': {}}},
 {'id': 1,
  'props': {'identity': 1,
   'start': 2,
   'end': 0,
   'type': 'Acquired',
   'properties': {'date': '11 Jan 1811', 'amount': '$23M'}}}]

In [30]:
# Clean up empty props, and add in corresponding EdgeIDs
ashok_edge_props = [p for p in ashok_edge_props if p['props']['properties']]
print(len(ashok_edge_props))
ashok_edge_props[:3]

5


[{'id': 1,
  'props': {'identity': 1,
   'start': 2,
   'end': 0,
   'type': 'Acquired',
   'properties': {'date': '11 Jan 1811', 'amount': '$23M'}}},
 {'id': 20,
  'props': {'identity': 20,
   'start': 26,
   'end': 0,
   'type': 'Acquired',
   'properties': {'date': '11-11-1999', 'amount': '$11M'}}},
 {'id': 2,
  'props': {'identity': 2,
   'start': 0,
   'end': 3,
   'type': 'Founded',
   'properties': {'Role': 'Chief Scientist'}}}]

In [39]:
edge_ids = []
properties = []
values = []
for e in ashok_edge_props:
    props = e['props']['properties']
    mask = ((connectivity_df[connectivity_mask].origin == 'ashok_edges') 
            & (connectivity_df[connectivity_mask].originalIndex == int(e['id'])))
    if np.sum(mask) == 0:
        # We dropped this from the output edge list, so skip it
        continue
    edge_id = connectivity_df[connectivity_mask][mask].index.values[0]
    edge_ids.extend([edge_id]*len(props))
    properties.extend(props.keys())
    values.extend(props.values())
edge_props = pd.DataFrame(data={"EdgeID": edge_ids, "Property": properties, "Value": values})
edge_props.head()

,EdgeID,Property,Value
0,278,Role,Chief Scientist
1,279,Role,Sales
2,279,Age,67Yrs
3,297,Role,WEB
4,297,Since,11-11-1999


In [41]:
edge_props.to_csv('5tables/EdgeProperty.tsv', index=False, sep='\t')

# Node properties

This section creates the node properties table.

Node properties extracted from existing graph with:

    MATCH (a) RETURN a as node

In [90]:
with open("ashok_node_properties.json", "rb") as f:
    ashok_node_props = json.loads(f.read())
print(len(ashok_node_props))
ashok_node_props[0]

32


{'node': {'identity': 0,
  'labels': ['Company'],
  'properties': {'Currentstatus': 'Active',
   'Founded': 'December 28, 2002; 16 years ago (2002-12-28)Mountain View, California, U.S.',
   'Headquarters': 'Sunnyvale, California, U.S.',
   'Alexarank': '58 (November 2019[update])[1]',
   'Employees': '15,000 (2019)',
   'Website': 'www.linkedin.com',
   'Launched': 'May 5, 2003; 16 years ago (2003-05-05)',
   'Industry': 'Internet',
   'Areaserved': 'Worldwide',
   'name': 'LinkedIn',
   'Typeofbusiness': 'Subsidiary',
   'Advertising': 'Google, AdSense',
   'Availablein': 'Multilingual (24)'}}}

In [91]:
mike_node_props = pd.read_csv('../scraping/node_properties.tsv', sep='\t', index_col=False)
print(mike_node_props.shape)
mike_node_props.head()

(905, 3)


,nodes,props,values
0,1&1 Drillisch,Founded,1983
1,1&1 Drillisch,Headquarters,Germany
2,1&1 Drillisch,Headquarters,Maintal
3,1&1 Drillisch,Industry,Telecommunications
4,1&1 Drillisch,Number of employees,"3,142"


In [92]:
nodes = []
properties = []
values = []
for node in ashok_node_props:
    nodeID = node['node']['properties']['name']
    props = [p for p in node['node']['properties'].keys() if p != 'name']
    vals  = [v for p, v in node['node']['properties'].items() if p != 'name']
    N = len(props)
    nodes.extend([nodeID] * N)
    properties.extend(props)
    values.extend(vals)
node_properties_df = pd.DataFrame(data={"NodeID": nodes,
                                        "Property": properties,
                                        "Value": values})
print(node_properties_df.shape)
node_properties_df.head()

(50, 3)


,NodeID,Property,Value
0,LinkedIn,Currentstatus,Active
1,LinkedIn,Founded,"December 28, 2002; 16 years ago (2002-12-28)Mo..."
2,LinkedIn,Headquarters,"Sunnyvale, California, U.S."
3,LinkedIn,Alexarank,58 (November 2019[update])[1]
4,LinkedIn,Employees,"15,000 (2019)"


In [93]:
node_properties_df = node_properties_df[node_properties_df.NodeID.isin(NodeLabels_df.term.values)]
print(node_properties_df.shape)

(40, 3)


In [94]:
mike_mask = mike_node_props.nodes.isin(NodeLabels_df.term.values)
print("% props in NodeLabels.tsv: {}%".format(np.sum(mike_mask) / len(mike_mask) * 100))

% props in NodeLabels.tsv: 73.25966850828729%


In [95]:
# Collapse values with the same props name into a list
grouped_props = mike_node_props.groupby(['nodes', 'props'])\
    .apply(lambda row: str(list(x for x in row['values'].values)))\
    .reset_index()
print(grouped_props.shape)
grouped_props.head(10)

(523, 3)


,nodes,props,0
0,1&1 Drillisch,Founded,['1983']
1,1&1 Drillisch,Headquarters,"['Germany', 'Maintal']"
2,1&1 Drillisch,Industry,['Telecommunications']
3,1&1 Drillisch,Number of employees,"['3,142']"
4,1&1 Drillisch,Operating income,['€567.2 million']
5,1&1 Drillisch,Revenue,['€3.663 billion']
6,1&1 Drillisch,Total assets,['€5.247 billion']
7,1&1 Drillisch,Type,['Aktiengesellschaft']
8,1&1 Drillisch,Website,['www']
9,2012 LinkedIn hack,Location,['Globally']


In [96]:
# Append to the node properties data frame.
# Shows the lengths along the way to illustrate how many properties were kept
print(len(node_properties_df))
node_properties_df = node_properties_df.append(
    pd.DataFrame(data={'NodeID': grouped_props.nodes,
                       'Property': grouped_props.props,
                       'Value': grouped_props[0]}))
print(len(node_properties_df))
node_properties_df = node_properties_df[node_properties_df.NodeID.isin(NodeLabels_df.term.values)]
print(len(node_properties_df))
node_properties_df.tail()

40
563
430


,NodeID,Property,Value
511,iPhone,Release date,"['11', '11 Pro / 11 Pro Max', '1st gen', '2007..."
512,iPhone,Type,['Smartphone']
513,iPhone,Website,['apple']
517,identi.ca,License,['Apache Licence 2.0']
518,identi.ca,Website,['identi']


In [ ]:
node_properties_df.to_csv('5tables/NodeProperty.tsv', sep='\t', index=False)